In [1]:
import pandas as pd
# module to read the contents of the file from a csv file
 
from contextlib import redirect_stdout
# module to redirect the output to a text file
from nltk.tokenize import word_tokenize
import re
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer,ISRIStemmer,PorterStemmer,SnowballStemmer

In [2]:
terms = []
# list to store the terms present in the documents
 
keys = []
# list to store the names of the documents
 
vec_Dic = {}
# dictionary to store the name of the document and the boolean vector as list
 
dicti = {}
# dictionary to store the name of the document and the terms present in it as a
# vector
 
dummy_List = []
# list for performing some operations and clearing them

In [3]:
def StopWords_Remove(document_tokenize):
    '''
    flage==1 english document
    flage==0 arabic document
    document_tokenize mean document performed tokenize
    '''
    arabic_stop=set(stopwords.words('arabic'))
    return [i for i in document_tokenize if i not in arabic_stop]   

In [4]:
def tokenize(document):
    '''
    flage==1 mean tokenize using word_tokenize
    flage==0 mean tokenize using split
    document input doctument text
    '''
    return word_tokenize(document)

In [5]:
def Arabic_text_preprocessing(text,flage=3):
    '''
    Flage Number
    1: ISRIStemmer
    2: SnowballStemmer
    3: WordNetLemmatizer
    4: POS
    '''
    text1=text
    #The regular expression [^\w\s] is used to match any character that is not a word character (\w) or a whitespace character (\s).
    text = re.sub(r'[^\w\s]', '',text)
    #splitting sentence into tokens
    text=word_tokenize(text.lower())
    #remove stopwords
    text=StopWords_Remove(text)
    #stemming of each word
    if flage<=3:
        if flage==1:
            stem=ISRIStemmer()
            text=[stem.stem(i) for i in text]
        elif flage==2:
            stem=SnowballStemmer('arabic')
            text=[stem.stem(i) for i in text]
        else:
            #Lemmatizer of each word
            lemmatizer=WordNetLemmatizer()
            text=[lemmatizer.lemmatize(i) for i in text]
    else:
        #part of speech of each word
        text=nltk.pos_tag(text)
    return text

In [6]:
def filter(documents, rows, cols):
    ''' create a dictionary which has the name of the document as key and the terms present in
    it as the list of strings  which is the value of the key'''
 
    for i in range(rows):
        for j in range(cols):
            # traversal through the data frame
 
            if(j == 1):
                # first column has the name of the document in the csv file
                keys.append(documents.iloc[i]['doc id'])
                #iat [] method is used to return data in a dataframe at the passed location.
                #loc[]accesses a group of rows and columns by label(s) or a boolean array
            else:
                for term in Arabic_text_preprocessing(documents.iloc[i,j]):
                    dummy_List.append(term)
                # dummy list to update the terms in the dictionary
                    if term not in terms:
                        # add the terms to the list if it is not present else continue
                        terms.append(term)
 
        copy = dummy_List.copy()
        # copying the dummy list to a different list
 
        dicti.update({documents.iloc[i]['doc id']: copy})
        # adding the key value pair to a dictionary
 
        dummy_List.clear()
        # clearing the dummy list

In [7]:
def bool_Representation(dicti, rows, cols):
    '''In this function we get a boolean representation of the terms present in the
    documents in the form of lists'''
 
    terms.sort()
    # we sort the elements in the alphabetical order for the convience
 
    for i in (dicti):
        # for every document in the dictionary we check for each string present in
        # the list
 
        for j in terms:
            # if the string is present in the list we append 1 else we append 0
 
            if j in dicti[i]:
                dummy_List.append(1)
            else:
                dummy_List.append(0)
            # appending 1 or 0 for obtaining the boolean representation
 
        copy = dummy_List.copy()
        # copying the dummy list to a different list
 
        vec_Dic.update({i: copy})
        # adding the key value pair to a dictionary
 
        dummy_List.clear()
        # clearing the dummy list


In [8]:
def Prediction2(query,documents):
    query = query.split()
    frist= documents[query[0]]
    for i in range(1,len(query)):
        frist = frist & df[query[i]]
        print(query[i])
    return frist

In [9]:
def query_Vector(query):
    '''In this function we represent the query in the form of boolean vector'''
 
    qvect = []
    # query vector which is returned at the end of the function
 
    for i in terms:
        # if the word present in the list of terms is also present in the query
        # then append 1 else append 0
 
        if i in query:
            qvect.append(1)
        else:
            qvect.append(0)
 
    return qvect
    # return the query vector which is obtained in the boolean form

In [10]:
def prediction(df, query):
    '''In this function we make the prediction regarding which document is related
    to the given query by performing the boolean operations'''
    query = Arabic_text_preprocessing(query)
    first = None
    for i in range(len(query)):
        try:
            output = list(df[query[i]].values) 
        except:
            output = []
        if len(output) > 0:
            if first is None:
                first = output
            else:
                copy = []
                for i in range(len(first)): 
                    if first[i] == 1 and output[i] == 1:
                        copy.append(1)
                    else:
                        copy.append(0)
                first = copy.copy()
    output=[]
    for i in range(len(first)):
        if first[i]==1:
            output.append(i+1)
    return output

In [11]:
def main(query,documents):
    #documents = pandas.read_csv(r'docs.csv')
    # to read the data from the csv file as a dataframe
 
    rows = len(documents)
    # to get the number of rows
 
    cols = len(documents.columns)
    # to get the number of columns
 
    filter(documents, rows, cols)
    # function call to read and separate the name of the documents and the terms
    # present in it to a separate list  from the data frame and also create a
    # dictionary which has the name of the document as key and the terms present in
    # it as the list of strings  which is the value of the key
 
    bool_Representation(dicti, rows, cols)
    df = pd.DataFrame(vec_Dic).transpose()
    df.columns= terms
    # In this function we get a boolean representation of the terms present in the
    # documents in the form of lists, later we create a dictionary which contains
    # the name of the documents as key and value as the list of boolean values
    #representing the terms present in the document
    # to get the query input from the user, the below input is given for obtaining
    # the output as in output.txt file
    # hockey is a national sport
    # splitting the query as a list of strings
    return prediction(df,query)

In [12]:
documents = pd.read_csv('documents.csv',usecols=[1,2])
documents.head()

,Documents,doc id
0,في عصرنا هذا، تحيط تطبيقا ُت الذكاء االصطناعي ...,document1
1,تُع ر المؤلفات الذكاء االصطناعي، على أنه» :درا...,document2
2,الذكاء االصطناعي يمكن تقسيمه إلى:\nالذكاء االص...,document3
3,أدوات بحث الذكاء االصطناعي \nخالل خمسين سنة من...,document4
4,لقد أصبح الذكاء االصطناعي مصطل ًحا شامًًل للتط...,document5


In [25]:
query='علم الذكاء الاصطناعي'

In [ ]:
main(query,documents)

In [14]:
query='الطالب و يذاكر'

In [15]:
main(query,documents)

['document14', 'document4', 'document3', 'document1']

In [16]:
list(documents['Documents'][documents['doc id']=='document4'].values)[0]

'أدوات بحث الذكاء االصطناعي \nخالل خمسين سنة من البحوث، صمم الذكاء االصطناعي عددا كبيرا من الوسائل لحل أصعب المشاكل في علوم الكمبيوتر. نناقش أدناه عدد \nقليل من أعم هذه الوسائل.\nالبحث والتحسين \nالمقالة الرئيسة :رياضيات االستمثال\nيمكن حل العديد من مشاكل الذكاء االصطناعي من الناحية النظرية بالبحث الذكي في العديد من الحلول الممكنة: يمكن أن يختزل التفكير \nالمنطقي إلى إجراء البحث. على سبيل المثال، يمكن اعتبار الدليل المنطقي بحثا عن مسار ينطلق من افتراضات إلى نتائج، حيث كل خطوة هي\nتطبيق لقاعدة االستدالل.]93[ تبحث الخوارزميات التخطيطية خالل تفريعات من األهداف الرئيسية والفرعية، في محاولة إليجاد الطريق إلى \nالهدف، وهي عملية تسمى تحليل الوسائل والغايات.]94[ تستخدم الخوارزميات الروبوتية محركات بحث محلية لتحريك األطراف واستيعاب \nاألشياء في مساحة التكوين.]64[ العديد من خوارزميات التعلم تستخدم خوارزميات البحث على أساس قابلية التحسين.\nأبحاث بسيطة شاملة]95[ نادرا ما تكون كافية لمعظم مشاكل العالم الحقيقي: فضاء البحث )عدد أماكن البحث( لتنمو بسرعة إلى أرقام فلكية. \nوالنتيجة هي بحثا بطيئا للغاية

In [34]:
df = pd.DataFrame(vec_Dic).transpose()
df.columns= terms
df

,0,1,10,100,102,104136,105,112,113,114,...,يوجه,يود,يوضح,يوظفوا,يوفر,يوقد,يوقع,يولد,يوم,يومنا
document1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
document2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
document3,0,0,0,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,0
document4,1,1,0,0,1,1,1,1,1,1,...,0,0,0,0,1,0,0,0,0,0
document5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
document6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
document7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
document8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
document9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
document10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
query=['الاصطناعي']
df[[query[0]]].values

array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1]], dtype=int64)